<a href="https://colab.research.google.com/github/Garimagupta85/Default-Prediction/blob/main/EDA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

## Loading Data 

## Preprocessing the data

## EDA

### Feature correlation in default class

### Feature correlation in paid class

### Correlation between different variable types with themselves

### Dependency of different variable types on payment variables